In [1]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.types import StringType


os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.hadoop:hadoop-aws:3.2.2,io.delta:delta-core_2.12:1.1.0  pyspark-shell "

os.environ['MINIO_USERNAME'] = 'grupo-02'
os.environ['MINIO_PASSWORD'] = '6HgSzdwj8eNpHcux'
        
config = {
    "spark.jars.packages":"org.apache.hadoop:hadoop-aws:3.2.2",
    "spark.kubernetes.namespace": "spark",
    "spark.kubernetes.container.image": "cronosnull/abd-spark-base:202301",
    "spark.executor.instances": "1",
    "spark.executor.memory": "10g",
    "spark.executor.cores": "1",
    "spark.driver.memory":"2g",
    "spark.driver.port":"38890",
    "spark.driver.blockManager.port":"7778",
    "spark.driver.bindAddress": "0.0.0.0",
    "spark.driver.host": "172.24.99.147",
    "spark.kubernetes.executor.request.cores":"500m",
    "spark.hadoop.fs.s3a.endpoint": "http://172.24.99.18:9000",
    # Credenciales de MinNIO, no olvide asignar las variables de entorno  
    "spark.hadoop.fs.s3a.access.key": os.environ.get('MINIO_USERNAME', "--"),
    "spark.hadoop.fs.s3a.secret.key": os.environ.get('MINIO_PASSWORD', "--"),
    "spark.hadoop.fs.s3a.path.style.access": True,
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    "spark.kubernetes.local.dirs.tmpfs":True,

}

def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster("k8s://https://172.24.99.68:16443")
    for key, value in config.items():
        conf.set(key, value)
    conf.set("spark.ui.port","4042")
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()

In [2]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 1.8.0_322
Branch 
Compiled by user  on 2022-03-26T09:34:47Z
Revision 
Url 
Type --help for more information.


In [3]:
spark = get_spark_session("grupo-02-Tarea1-Punto1", SparkConf())
spark

In [4]:
spStopWords = spark.read.text("s3a://user-data/grupo-02/stopwords/spanish")

In [5]:
columns = spStopWords.select("*").toPandas()
col1 = list(columns['value'])

In [ ]:
initial = "s3a://noticias2016/individual_files/news_"
final = ".json.gz"
listpaths = []
for i in range(1,175000):
    numberString = str(i).zfill(7)
    result = initial + numberString + final
    listpaths.append(result)


In [ ]:
df = spark.read.json("s3a://noticias2016/individual_files/news_0000001.json.gz")
schema = df.schema
test_df = spark.read.json(listpaths, schema=schema)

In [6]:
df = spark.read.json("s3a://noticias2016/individual_files/news_0000001.json.gz")
schema = df.schema
test_df = spark.read.json("s3a://noticias2016/individual_files/*.json.gz", schema=schema)

In [ ]:
rows = test_df.count()
print(f"DataFrame Rows count : {rows}")

# Parte 1

## Punto B - Word Frequency Title

In [ ]:
df_title = test_df.select("title")

In [ ]:
rdd_title = df_title.rdd.map(lambda x: [x[0]])

In [ ]:
rdd_title.take(5)

In [ ]:
import re
rdd = (
    rdd_title.flatMap(lambda line: re.split(r"\W+", str(line).lower()))
    .filter(lambda z: re.match(r"\b[^\W\d_]+\b",z))
    .filter(lambda x: x!= "")
    .filter(lambda y: y not in col1)
    .map(lambda w: (w, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda p: p[1])
)

In [ ]:
countWords = rdd.toDF(["word","word_freq"])

In [ ]:
countWords.tail(10)

## Punto C - Word Frequency Body

In [ ]:
df = spark.read.json("s3a://noticias2016/individual_files/news_0000001.json.gz")

In [ ]:
df.show()

In [7]:
df_body = test_df.select("text")

In [8]:
rdd_body = df_body.rdd.map(lambda x: [x[0]])

In [9]:
import re
rdd_parte1C = (
    rdd_body.flatMap(lambda line: re.split(r"\W+", str(line).lower()))
    .filter(lambda z: re.match(r"\b[^\W\d_]+\b",z))
    .filter(lambda x: x!= "")
    .filter(lambda y: y not in col1)
    .map(lambda w: (w, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda p: p[1])
)

In [10]:
countWordsBody = rdd_parte1C.toDF(["word","word_freq"])

In [11]:
countWordsBody.tail(10)

[Row(word='octubre', word_freq=135809),
 Row(word='año', word_freq=147180),
 Row(word='gobierno', word_freq=147344),
 Row(word='nla', word_freq=163562),
 Row(word='ser', word_freq=170908),
 Row(word='dos', word_freq=193592),
 Row(word='si', word_freq=206173),
 Row(word='años', word_freq=215962),
 Row(word='nel', word_freq=261643),
 Row(word='n', word_freq=370688)]